In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import open3d as o3d


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
camera_intrinsics = np.array([
    [577.590698, 0.000000, 318.905426 ,0.000000],
    [0.000000 ,578.72979,242.683609 ,0.000000],
    [0.000000, 0.000000, 1.000000 ,0.000000],
    [0.000000 ,0.000000, 0.000000 ,1.000000]
    ],dtype='float64')
fx = camera_intrinsics[0][0]
fy = camera_intrinsics[1][1]
S = camera_intrinsics[0][1]
cx = camera_intrinsics[0][2]
cy = camera_intrinsics[1][2]

camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(width=640, height=480, fx=fx,fy=fy,cx=cx,cy=cy)

In [3]:
scene = "scene0005_01"

In [17]:
scans_list = os.listdir('./imgs/'+scene)
scans_list = list(map(lambda x: x.split('.')[0],scans_list))

pcd_list = []
cnt = 0
for i,scan in enumerate(scans_list):
    depth_map = o3d.geometry.Image(np.ascontiguousarray(np.load('./depths/'+scene+"/"+scan+'.npy')).astype(np.float32))
    rgb_img = cv2.resize(cv2.imread('./imgs/'+scene+'/'+scan+'.png'),(640,480))
    rgb_img = rgb_img//2
    mask = cv2.resize(cv2.imread('./masks/'+scene+'/'+scan+'.png'),(640,480))
    floor = np.isin(mask[:,:,0],[5])
    rgb_img[floor,1] *= 2
    rgb_img[~floor,0] *= 2
    rgb_img = o3d.geometry.Image(rgb_img)
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(o3d.geometry.Image(rgb_img), o3d.geometry.Image(depth_map),convert_rgb_to_intensity=False)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, camera_intrinsic)
    pose = np.loadtxt('./poses/'+scene+'/'+scan+'.txt',dtype=np.float32)
    pcd.transform(pose)
    pcd_list.append(pcd.uniform_down_sample(100))



In [18]:
o3d.visualization.draw_geometries(pcd_list)

In [37]:
print(np.asarray(pcd_list[5].points)[150])

[1.34161873 6.60764183 2.28571952]


In [47]:
all_points = np.asarray(pcd_list[0].points)
all_colors = np.asarray(pcd_list[0].colors)
for pcd in pcd_list[1:]:
    all_points = np.vstack((all_points,np.asarray(pcd.points)))
    all_colors = np.vstack((all_points,np.asarray(pcd.colors)))
all_points = o3d.utility.Vector3dVector(all_points)
all_colors = o3d.utility.Vector3dVector(all_colors)

In [49]:
combined_pcd = o3d.geometry.PointCloud()
combined_pcd.points = all_points
combined_pcd.colors = all_colors
o3d.visualization.draw_geometries([combined_pcd])

In [52]:
print(np.asarray(combined_pcd.colors)[1500])

[4.06269268 5.80506467 1.1378507 ]


In [20]:
o3d.io.write_point_cloud('./pcds/'+scene+'.pcd',)

TypeError: write_point_cloud(): incompatible function arguments. The following argument types are supported:
    1. (filename: str, pointcloud: open3d.cpu.pybind.geometry.PointCloud, write_ascii: bool = False, compressed: bool = False, print_progress: bool = False) -> bool

Invoked with: './pcds/scene0005_01.pcd', std::vector<Eigen::Vector3d> with 8395728 elements.
Use numpy.asarray() to access data.

Did you forget to `#include <pybind11/stl.h>`? Or <pybind11/complex.h>,
<pybind11/functional.h>, <pybind11/chrono.h>, etc. Some automatic
conversions are optional and require extra headers to be included
when compiling your pybind11 module.

#### Occupancy Map Creation

In [36]:
from PIL import Image as im
grid_width  = 500
grid_height = 700
mpixel = 0.05

occupancy_grid = np.zeros((grid_width,grid_height))
accumulation_grid = np.zeros((grid_width,grid_height))
free_grid = np.zeros((grid_width,grid_height))  
occupied_image = np.zeros((occupancy_grid.shape[0],occupancy_grid.shape[1],3))
free_image = np.zeros((occupancy_grid.shape[0],occupancy_grid.shape[1],3))
for points in pcd_list:
    points = np.asarray(points.points)
    for point in points:
        if(point[2] > 0.5):
            free_grid[int(point[1]/mpixel+grid_width/2)][int(point[0]/mpixel+grid_height/2)] +=1 
#             continue

threshold = 1

for i in range(grid_width):
    for j in range(grid_height):
        if(free_grid[i][j]>=threshold):
            occupied_image[i][j] = np.array([255,255,255])
            free_image[i][j] = np.array([0,0,0])
        else:
            occupied_image[i][j] = np.array([0,0,0])
            free_image[i][j] = np.array([255,255,255])

data = im.fromarray(occupied_image.astype(np.uint8))
data = data.convert("RGB")
data.save('./Results/'+scene+'_obstacle.png')
            
data = im.fromarray(free_image.astype(np.uint8))
data = data.convert("RGB")
data.save('./Results/'+scene+'_free.png')
